# Abstrakt


# Cel projektu

Celem projektu było stworzenie liniowego modelu ilości wpadków na drogach z pomocą zmiennych objaśniających pobranych ze strony policji oraz na podstawie pogody.

In [ ]:
# Dane